# 머신러닝

- 이미지를 콘텐츠 기반 추천 시스템에 적용하기 위해 코사인 유사도 모델 활용
- 운동 이름, 부위, 방법에 따를 코사인 유사도를 계산
- 유사도가 높은 순서대로 운동 기구를 정렬하고, 사용자에게 가장 적합한 운동기구 추천 및 운동방법 제시
- 문서를 벡터 표현으로 변경하는 방법
    - 1) TF - IDF : 단어의 빈도 X 문서 빈도의 역수
    - 2) CountVectorizer : 단어의 빈도 확인

In [2]:
import pandas as pd

exercise = pd.read_csv("team_project_django/exercise/machine_learning/exercise.csv", encoding='CP949')

exercise.head()

,ex_name,ex_part,ex_method,ex_video1,ex_video2
0,백 익스텐션,허리,발을 지지대에 고정하고 허벅지 앞부분이 패드에 닿게 엎드린다. 팔은 가슴 위에 모으...,back_extension.mp4,https://www.youtube.com/embed/pCY8xvkLHn8?si=-...
1,데드리프트 - 바벨,허리,"둔근, 대퇴 후면 근육 등 전신 근육 발달에 중요한 영향을 끼치는 운동이다. 신체의...",dead_lift-barbell.mp4,https://www.youtube.com/embed/EBjYQeeBI-0?si=O...
2,데드리프트 - 덤벨,허리,"둔근, 대퇴 후면 근육 등 전신 근육 발달에 중요한 영향을 끼치는 운동이다. 신체의...",dead_lift-dumbbell.mp4,https://www.youtube.com/embed/7RN8DHl6r3A?si=w...
3,굿모닝,허리,상체를 숙여 인사하는 듯한 동작 때문에 ‘굿모닝’이라고 이름 붙여졌다. 맨손으로 손...,good_morning.mp4,https://www.youtube.com/embed/e1bCOy0S0SU?si=o...
4,굿모닝 - 바벨,허리,등과 허리를 강화시키고 둔근과 대퇴이두근을 발달시키는 데 효과적인 운동이다. 무거운...,good_morning-barbell.mp4,https://www.youtube.com/embed/WwhCnOfl9e0?si=A...


### 1. TF - IDF (단어 빈도, 문서 빈도의 역수 곱)

- 단어 간 중요도에 따라 가중치 부여

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(exercise['ex_method'])
print("TF-IDF 행렬의 크기 : ", tfidf_matrix.shape)

TF-IDF 행렬의 크기 :  (52, 1713)


In [5]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print("코사인 유사도 연산 결과 : ", cosine_sim.shape)

코사인 유사도 연산 결과 :  (52, 52)


In [6]:
title_to_index = dict(zip(exercise['ex_name'], exercise.index))

# 굿모닝의 인덱스를 리턴
idx = title_to_index['굿모닝']
print(idx)

3


In [7]:
def get_recommendations(name, cosine_sim=cosine_sim):
    
    idx = title_to_index[name]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[0:7]
    movie_indices = [idx[0] for idx in sim_scores]
    
    return exercise.iloc[movie_indices]

In [8]:
get_recommendations('아놀드 프레스')

,ex_name,ex_part,ex_method,ex_video1,ex_video2
23,아놀드 프레스,어깨,아놀드 슈왈제네거가 삼각근 운동으로 전면 삼각근의 수축으로 시작하여 측면 삼각근 ...,arnold_press.mp4,https://www.youtube.com/embed/PPy2WOkiGjw?si=I...
22,숄더 프레스 - 덤벨,어깨,삼각근의 전면과 측면의 볼륨과 선명도를 증가시킬 수 있는 운동이다. 바벨 프레스에 ...,shoulder_press-dumbbell.mp4,https://www.youtube.com/embed/Ech4wjq4tWA?si=Z...
46,컬 프레스 - 덤벨,팔,덤벨 컬과 덤벨 숄더 프레스 운동을 결합한 형태의 운동으로 상완과 동원되어 이루어...,curl_press-dumbbell.mp4,https://www.youtube.com/embed/Ia9DYFMkMmU?si=I...
47,"컬 - 덤벨, 인클라인",팔,경사진 벤치에 누워서 실시하는 운동이다. 상완이두근의 위쪽 부위를 더욱 자극하고 선...,curl-dumbbell.mp4,https://www.youtube.com/embed/VhiBywqdGmg?si=4...
24,프런트 레이즈 - 덤벨,어깨,삼각근의 선명도를 높일 수 있는 단관절 운동 중 전면 삼각근을 발달시키기 위한 운동...,front_raises-dumbbell.mp4,https://www.youtube.com/embed/m0ddyws4VL4?si=Y...
44,"로우 - 덤벨, 원 암",등,몸의 중심선(척추)에 가까운 등의 근육을 집중적으로 발달시키고 선명도를 증가시킬 수...,"row-dumbbell,one_arm.mp4",https://www.youtube.com/embed/2i1v1UvsP_w?si=u...
50,"트라이셉스 익스텐션 - 덤벨, 라잉",팔,팔 뒤쪽 상완삼두근의 근력을 증가시키는 운동으로 삼두근의 안쪽 모양을 선명하게 . ...,"triceps_extension-dumbbell,_lying.mp4",https://www.youtube.com/embed/uZOwQLaq0-g?si=V...


### 2. CountVector (단어 빈도)

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [10]:
# 운동 이름 데이터의 텍스트 피쳐 벡터라이징
count_vect_name = CountVectorizer()
exercise_name = count_vect_name.fit_transform(exercise['ex_name'])

# 운동 이름 데이터의 코사인 유사도
name = cosine_similarity(exercise_name, exercise_name)

In [11]:
# 운동 부위 데이터의 텍스트 피쳐 벡터라이징
count_vect_part = CountVectorizer()
exercise_part = count_vect_part.fit_transform(exercise['ex_part'])

# 운동 부위 데이터의 코사인 유사도
part = cosine_similarity(exercise_part, exercise_part)

In [12]:
# 운동 방법 데이터의 텍스트 피쳐 벡터라이징
count_vect_method = CountVectorizer()
exercise_method = count_vect_method.fit_transform(exercise['ex_method'])

# 운동 방법 데이터의 코사인 유사도
method = cosine_similarity(exercise_method, exercise_method)

In [13]:
# 가중치 부여
exercise_weight = (
                    + name * 0.3
                    + part * 1
                    + method * 0.3
)

exercise_sorted_ind = exercise_weight.argsort()[:, ::-1]

In [14]:
def find_exercise_name(exercise, sorted_ind, name):
    
    name_title = exercise[exercise['ex_name'] == name]
    name_index = name_title.index.values
    similar_indexes = sorted_ind[name_index, :6]
    similar_indexes = similar_indexes.reshape(-1)
    
    return exercise.iloc[similar_indexes][:6]

find_exercise_name(exercise, exercise_sorted_ind, "점프 스쿼트")

,ex_name,ex_part,ex_method,ex_video1,ex_video2
11,점프 스쿼트,허벅지,하체의 체지방은 연소하고 탄력을 향상시키는 데 효과적인 운동이다. 자기 체중을 갖고...,jump_squat.mp4,https://www.youtube.com/embed/SL-reYwCRes?si=f...
6,스쿼트 - 스미스 머신,허벅지,바벨 스쿼트 운동에 비해 고립된 운동으로 대퇴부와 둔부 크기를 증가시키기 위한 운동...,squat-smith_machine.mp4,https://www.youtube.com/embed/xAzWNBmDS-I?si=L...
8,레그 익스텐션,허벅지,"허벅지 전면 근육을 발달시키는 운동이다. 여성의 경우, 적당한 무게로 횟수를 늘려 ...",leg_extension.mp4,https://www.youtube.com/embed/mS9iwXhycJI?si=7...
9,레그 컬 - 라잉,허벅지,대퇴 후면의 슬굴곡근을 발달시키기 위한 운동이다. 스탠딩 레그 컬보다 많은 중량을 ...,leg_curl-lying.mp4,https://www.youtube.com/embed/9w8d_EL-7e4?si=J...
10,"데드리프트 - 바벨, 스티프 레그드",허벅지,"슬굴곡근과, 척추기립근, 대둔근이 통합적으로 동원되어 대퇴 후면과 등 하부를 동시에...",dead_lift-barbell_stiff-legged.mp4,https://www.youtube.com/embed/8Pc-HiMTfD0?si=K...
7,레그 프레스,허벅지,대퇴사두근과 둔근을 강화시켜주는 대표적이고 필수적인 하체 운동이다. 초보자가 실시하...,leg_press.mp4,https://www.youtube.com/embed/EV0F_3S7Sks?si=w...


In [15]:
def find_exercise_part(exercise, sorted_ind, part):
    
    part_title = exercise[exercise['ex_part'] == part]
    part_index = part_title.index.values
    similar_indexes = sorted_ind[part_index, :6]
    similar_indexes = similar_indexes.reshape(-1)
    
    return exercise.iloc[similar_indexes][:6]

find_exercise_part(exercise, exercise_sorted_ind, "가슴")

,ex_name,ex_part,ex_method,ex_video1,ex_video2
30,펙덱 플라이 머신,가슴,운동을 처음 시작하는 초보자들이 덤벨 플라이를 실시하기 전에 근육의 자극을 경험하고...,pec_dec_fly_machine.mp4,https://www.youtube.com/embed/oJRW-Y1QvJQ?si=J...
35,플라이 - 덤벨,가슴,벤치에 앉아 팔을 가슴 옆에 붙이고 덤벨을 든다. 벤치에 누워 가슴 중앙과 덤벨이 ...,fly-dumbbell.mp4,https://www.youtube.com/embed/yVW8fgjlags?si=Q...
31,체스트 프레스 머신,가슴,가슴운동에서 제일 중요한 부분은 어깨 정렬이다. 대부분 가슴운동을 할 때 부상당하는...,chest_press.mp4,https://www.youtube.com/embed/voatYg6DOs0?si=I...
33,어시스트 치닝 앤 디핑 머신,"가슴, 팔",평행봉이나 디핑 바(Dipping Bar)에 두 팔을 편 채로 짚고서 체중을 지탱한...,chinning_dipping.mp4,https://www.youtube.com/embed/RQqXO3aNjxU?si=d...
36,벤치 프레스 - 덤벨,가슴,팔을 가슴 옆에 붙여 덤벨을 든다. 벤치에 누워 쇄골 아랫부분과 수직이 되게 덤벨을...,bench_press-dumbbell.mp4,https://www.youtube.com/embed/xTQL6jvVMNA?si=l...
32,케이블머신,가슴,케이블의 양쪽 손잡이를 잡고 기구 중앙에 서서 엉덩이 너비만큼 발을 벌린다. 무릎을...,cable_machine.mp4,https://www.youtube.com/embed/U3EN1caN-tk?si=n...


<hr>

## 머시러닝 모델 파일로 저장하기

In [19]:
# 훈련된 모델 파일로 저장하기
import joblib
joblib.dump(name, './exercise_name.pkl')
joblib.dump(part, './exercise_part.pkl')
joblib.dump(method, './exercise_method.pkl')
joblib.dump(exercise_weight, './exercise_weight.pkl')

['./exercise_weight.pkl']

In [20]:
# 저장한 모델 불러오기
import joblib
mac_name = joblib.load('./exercise_name.pkl')
mac_part = joblib.load('./exercise_part.pkl')
mac_method = joblib.load('./exercise_method.pkl')
mac_weight = joblib.load('./exercise_weight.pkl')

- 이름, 부위, 방법 3가지로 나누어 모델 파일저장 결과 정확도가 떨어짐을 확인.
- 가중치를 부여한 값에 대해 모델 파일 저장 결과 기본 결과와 비슷한 결과가 나옴 (장고에서 실행 시)

In [21]:
# 가중치 부여
exercise_weight = (
                    + mac_name * 0.3
                    + mac_part * 1
                    + mac_method * 0.3
)

exercise_sorted_ind = exercise_weight.argsort()[:, ::-1]

In [22]:
# 결과 예측하기
def find_exercise_name(exercise, sorted_ind, name):
    
    name_title = exercise[exercise['ex_name'] == name]
    name_index = name_title.index.values
    similar_indexes = sorted_ind[name_index, :6]
    similar_indexes = similar_indexes.reshape(-1)
    
    return exercise.iloc[similar_indexes][:6]

find_exercise_name(exercise, exercise_sorted_ind, "점프 스쿼트")

,ex_name,ex_part,ex_method,ex_video1,ex_video2
11,점프 스쿼트,허벅지,하체의 체지방은 연소하고 탄력을 향상시키는 데 효과적인 운동이다. 자기 체중을 갖고...,jump_squat.mp4,https://www.youtube.com/embed/SL-reYwCRes?si=f...
6,스쿼트 - 스미스 머신,허벅지,바벨 스쿼트 운동에 비해 고립된 운동으로 대퇴부와 둔부 크기를 증가시키기 위한 운동...,squat-smith_machine.mp4,https://www.youtube.com/embed/xAzWNBmDS-I?si=L...
8,레그 익스텐션,허벅지,"허벅지 전면 근육을 발달시키는 운동이다. 여성의 경우, 적당한 무게로 횟수를 늘려 ...",leg_extension.mp4,https://www.youtube.com/embed/mS9iwXhycJI?si=7...
9,레그 컬 - 라잉,허벅지,대퇴 후면의 슬굴곡근을 발달시키기 위한 운동이다. 스탠딩 레그 컬보다 많은 중량을 ...,leg_curl-lying.mp4,https://www.youtube.com/embed/9w8d_EL-7e4?si=J...
10,"데드리프트 - 바벨, 스티프 레그드",허벅지,"슬굴곡근과, 척추기립근, 대둔근이 통합적으로 동원되어 대퇴 후면과 등 하부를 동시에...",dead_lift-barbell_stiff-legged.mp4,https://www.youtube.com/embed/8Pc-HiMTfD0?si=K...
7,레그 프레스,허벅지,대퇴사두근과 둔근을 강화시켜주는 대표적이고 필수적인 하체 운동이다. 초보자가 실시하...,leg_press.mp4,https://www.youtube.com/embed/EV0F_3S7Sks?si=w...


In [23]:
def find_exercise_part(exercise, sorted_ind, part):
    
    part_title = exercise[exercise['ex_part'] == part]
    part_index = part_title.index.values
    similar_indexes = sorted_ind[part_index, :6]
    similar_indexes = similar_indexes.reshape(-1)
    
    return exercise.iloc[similar_indexes][:6]

find_exercise_part(exercise, exercise_sorted_ind, "가슴")

,ex_name,ex_part,ex_method,ex_video1,ex_video2
30,펙덱 플라이 머신,가슴,운동을 처음 시작하는 초보자들이 덤벨 플라이를 실시하기 전에 근육의 자극을 경험하고...,pec_dec_fly_machine.mp4,https://www.youtube.com/embed/oJRW-Y1QvJQ?si=J...
35,플라이 - 덤벨,가슴,벤치에 앉아 팔을 가슴 옆에 붙이고 덤벨을 든다. 벤치에 누워 가슴 중앙과 덤벨이 ...,fly-dumbbell.mp4,https://www.youtube.com/embed/yVW8fgjlags?si=Q...
31,체스트 프레스 머신,가슴,가슴운동에서 제일 중요한 부분은 어깨 정렬이다. 대부분 가슴운동을 할 때 부상당하는...,chest_press.mp4,https://www.youtube.com/embed/voatYg6DOs0?si=I...
33,어시스트 치닝 앤 디핑 머신,"가슴, 팔",평행봉이나 디핑 바(Dipping Bar)에 두 팔을 편 채로 짚고서 체중을 지탱한...,chinning_dipping.mp4,https://www.youtube.com/embed/RQqXO3aNjxU?si=d...
36,벤치 프레스 - 덤벨,가슴,팔을 가슴 옆에 붙여 덤벨을 든다. 벤치에 누워 쇄골 아랫부분과 수직이 되게 덤벨을...,bench_press-dumbbell.mp4,https://www.youtube.com/embed/xTQL6jvVMNA?si=l...
32,케이블머신,가슴,케이블의 양쪽 손잡이를 잡고 기구 중앙에 서서 엉덩이 너비만큼 발을 벌린다. 무릎을...,cable_machine.mp4,https://www.youtube.com/embed/U3EN1caN-tk?si=n...


<hr>